|ATLAS - Time Card Automation Program|
|-|
|Using GPT 3.5 Turbo|

In [8]:
# libs
import openai
import requests
import json

# keys
from config import OPENAI_API_TOKEN

In [ ]:
# KEY
openai.api_key = OPENAI_API_TOKEN
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {openai.api_key}"
}

def fix_grammar_spelling(text):
    prompt = f"Please correct any grammar and spelling mistakes in the following text:\n\n{text}\n\nCorrected Text:"
    response = openai.Completion.create(
        engine="text-davinci-002",
        prompt=prompt,
        max_tokens=150,
        n=1,
        stop=None,
        temperature=0.5,
    )
    return response.choices[0].text.strip()

def generate_accomplishments(upstream_description, daily_objectives):
    prompt = f"{upstream_description}\n\nDaily Objectives:\n\n{daily_objectives}\n\nDaily Accomplishments:"
    data = {
        "engine": "gpt-3.5-turbo",
        "messages": [{"role": "system", "content": prompt}],
        "max_tokens": 150,
        "temperature": 0.7,
    }

    response = requests.post(
        "https://api.openai.com/v1/chat/completions",
        headers=headers,
        data=json.dumps(data)
    )
    response_dict = response.json()
    return response_dict['choices'][0]['messages'][-1]['content'].strip()


upstream_description = '''Take a list of tasks as Daily Objectives. Pretend they are all completed. 
Then, write a list of Daily Accomplishments describing the completed tasks. 
Make sure the sentences are clear and have correct grammar. 
Feel free to be creative and descriptive with the Daily Accomplishments.
Perform sentence completion and add to descriptive text for Daily Accomplishments.
The output should be like below.

Daily Objectives:
-bullet 1
-bullet 2
-bullet 3

Daily Accomplishments:
-bullet 1
sub bullet 1
-bullet 2
sub bullet 3
-bullet 3

'''

# Collect user input for the 5 daily objectives
user_objectives = []
for i in range(1, 4):
    user_input = input(f"Enter daily objective {i}: ")
    corrected_input = fix_grammar_spelling(user_input)
    user_objectives.append(f"{i}. {corrected_input}")

daily_objectives = "\n".join(user_objectives)
print("\nDaily Objectives:\n")
print(daily_objectives)

accomplishments = generate_accomplishments(upstream_description, daily_objectives)
print("\nDaily Accomplishments:\n")
print(accomplishments)
